# AgeDB Data Preparation Notebook

The dataset is structured as follows:

```text
AgeDB:
|---0_MariaCallas_35_f.jpg
|---1_MariaCallas_40_f.jpg
|---...


The labels are contained within the name of the image file and is encoded as follows:
ImageID_PersonName_Age_Gender.jpg

The dataset will be organized such that all the images are contained within an Images folder and a labels.csv file will hold the labels.

In [1]:
import os, numpy as np, shutil, matplotlib.pyplot as plt